<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 128)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Contrastive loss to maximize similarity between positive pairs
def contrastive_loss(x1, x2, temperature=0.07):
    # Compute cosine similarity matrix
    similarity_matrix = F.cosine_similarity(x1.unsqueeze(1), x2.unsqueeze(0), dim=-1) / temperature
    # Create labels for positive pairs
    labels = torch.arange(x1.size(0)).to(x1.device)
    # Compute contrastive loss
    loss = F.cross_entropy(similarity_matrix, labels)
    return loss

model = SimpleCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example self-supervised training loop
for epoch in range(100):
    optimizer.zero_grad()
    x1, x2 = torch.randn(32, 3, 32, 32), torch.randn(32, 3, 32, 32)  # Positive pairs
    z1, z2 = model(x1), model(x2)
    loss = contrastive_loss(z1, z2)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')